# WB RecSys Project

# Общее описание проекта

Необходимо на основании взаимодействий пользователей с товарами предсказать следующие взаимодействия пользователей с товарами.

# Stage 3

- Сформировать обучающую выборку
- Спроектировать схему валидации с учетом специфики задачи
- Обосновать выбор способа валидации


# Preprocessing train_data

# Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import dill

from IPython.display import Image

import matplotlib.pyplot as plt

import seaborn as sns

# USE THIS STYLE
# plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
# 
# OR THIS STYLE
import aquarel

import warnings

warnings.filterwarnings("ignore")

theme = aquarel.load_theme("arctic_light")
theme.set_font(family="serif")
theme.apply()

### Путь до данных

In [ ]:
data_path = "../../data_closed/"

# train data

### Чтение 

In [ ]:
interactions_df = pd.read_parquet(data_path + "train_data_10_10_24_10_11_24_final.parquet")
display(interactions_df)
display(interactions_df.dtypes)

In [ ]:
# Отсортируем по дате
interactions_df = interactions_df.sort_values(by="dt")

In [ ]:
interactions_df["subject_id"].unique()

Сократим размерность, дропнув столбец subject_id (константное значение)

In [ ]:
interactions_df = interactions_df.drop(columns=["subject_id"])

Следовательно, можно заключить, что номер "69020" &mdash; это внутренняя кодировка для категории товара. Теперь появилось представление за что отвечает каждое поле в таблице: 


|    Поле    |                      Значение                      |
| :--------: | :------------------------------------------------: |
| wbuser_id  |                  id пользователя                   |
|   nm_id    |                     id товара                      |
| subject_id |                id категории товара                 |
|     dt     | дата и время взаимодействия пользователя с товаром |
|    date    |     дата взаимодействия пользователя с товаром     |



Значит, будет строиться модель для предсказания взаимодейсвия пользователя с конкретным товаром.



Для удобства переименуем колонки
- `wbuser_id` $\rightarrow$ `user_id`
- `nm_id` $\rightarrow$ `item_id`

In [ ]:
# Переименовываем колонки
interactions_df = interactions_df.rename(
    columns={
        "wbuser_id": "user_id",
        "nm_id": "item_id",
    }
)

Посмотрим в принципе на количество уникальных значений для столбцов user_id, item_id, для столбцов отвечающих за дату и время посмотрим промежутки за которые предоставлены данные.  

In [ ]:
interactions_df["user_id"].unique().shape

~ 4 млн пользователей 

In [ ]:
interactions_df["item_id"].unique().shape

~ 400 тыс. товаров 

In [ ]:
print(f"min_date = {interactions_df['dt'].min()}")
print(f"max_date = {interactions_df['dt'].max()}")

Данные из датасета собраны за два дня.

Дропнем столбец date, т.к. он по сути дублирует столбец dt

In [ ]:
interactions_df = interactions_df.drop(columns=["date"])

### Посмотрим на полноту данных (наличие NaN значений в таблице):

In [ ]:
interactions_df.isnull().any()

Все поля таблицы заполнены.

In [ ]:
# Посмотрим распределение покупок по часам для каждого из двух дней
day_1 = interactions_df["dt"][
    interactions_df["dt"].dt.day == 10
].dt.hour.to_frame()
day_2 = interactions_df["dt"][
    interactions_df["dt"].dt.day == 11
].dt.hour.to_frame()

day_1 = day_1["dt"].value_counts().to_frame().sort_values(by="dt").reset_index()
day_2 = day_2["dt"].value_counts().to_frame().sort_values(by="dt").reset_index()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 6), sharey=True)

sns.barplot(
    x=day_1["dt"],
    y=day_1["count"],
    ax=ax[0],
)
ax[0].set_title("Распределение заказов 10 числа")
ax[0].set_xlabel("Время, часы")
ax[0].set_ylabel("Число заказов")

sns.barplot(
    x=day_2["dt"],
    y=day_2["count"],
    ax=ax[1],
)
ax[1].set_title("Распределение заказов 11 числа")
ax[1].set_xlabel("Время, часы")
ax[1].set_ylabel("Число заказов")


plt.show()

## 1. Разделим данные на train \ test

In [ ]:
Image("../../stage_2/images/validation.png")

Под test выделим ~15%-20% от имеющихся данных: данные предоставлены за 2 дня, так что в тест пойдут данные за последние ~8 часов.
Под обучение модели второго уровня выделим 1/5 от данных идущих на train, т.е. ~8 часов.

Плюс в копилку выбора такого разделения &mdash; это активность пользователей и заказов: 
активность большая активность начинается с 8.00 часов (что совпадает с началом разбиения ranker), 
и +- одинакова от момента начала разбиения для ранкера и до конца test разбиения.

In [ ]:
# Конечная дата
max_date = interactions_df["dt"].max()

# Дата начала данных для теста
train_test_sep = max_date - pd.Timedelta(hours=8)

# Данные для теста
test_df = interactions_df[(interactions_df["dt"] >= train_test_sep)]

# Данные для обучения моделей первого 
# и второго уровня (разделение будет потом)
train_df = interactions_df[(interactions_df["dt"] < train_test_sep)]

Сохраним в бинарник

In [ ]:
with open(data_path + "train_df.dill", "wb") as f:
    dill.dump(train_df, f)

### Соберем test_df

Соберем следующие просмотренные товары в списки

In [ ]:
# Таблица взаимодействий уже была заранее отсортирована по дате,
# так что порядок взаимодействий по дате сохранится 
test_df = (
    test_df.groupby("user_id", as_index=False)
    .agg({"item_id": list})
)

test_df

Теперь следующий вопрос: сколько товаров рекомендовать? 

In [ ]:
# Информации о длинах интеракций пользователей в test_df
test_df["item_id"].apply(len).describe()

Будем рекомендовать следующие 10 позиций для пользователя.
Для этого модифицируем  `test_df["item_id"]`

In [ ]:
def format_item_id_test_df(x):
    # просто добъем количество айтемов в списке до 10
    # если меньше чем 10, то будем повторять список
    while len(x) < 10:
        x += x
    return x[:10]
    
test_df["item_id"] = test_df["item_id"].apply(format_item_id_test_df)

test_df

Сохраним в бинарник

In [ ]:
with open(data_path + "test_df.dill", "wb") as f:
    dill.dump(test_df, f)

## 2. Модифицируем только таблицу train_df


In [ ]:
# Загрузим таблицу 
with open(data_path + "train_df.dill", "rb") as f:
    train_df = dill.load(f)

train_df

Составим веса weight (рейтинг конкретных товаров для пользователя)

In [ ]:
# Посчитаем количество взаимодействий пользователя
# с каждым конкретным товаром
train_df_weights = (
    train_df.groupby(["user_id", "item_id"])
    .agg(
        {
            "item_id": "count",
        }
    )
    .rename(
        columns={
            "item_id": "ui_inter",
        }
    )
    .reset_index()
)

display(train_df_weights)

Посчитаем количество всех взаимодейстий пользователя (u_total_inter)
и поделим на полученное значение число взаимодействий с каждым конкретным товаром (ui_inter)

In [ ]:
total_users_interactions_count = (
    train_df_weights[["user_id", "ui_inter"]]
    .groupby("user_id")
    .sum()
    .rename(
        columns={
            "ui_inter": "u_total_inter",
        }
    )
)

display(total_users_interactions_count)

In [ ]:
# Соединим таблицы
train_df_weights = train_df_weights.join(
    total_users_interactions_count,
    on="user_id",
    how="left",
)

# Рассчитаем веса товаров
train_df_weights["weight"] = train_df_weights["ui_inter"] / train_df_weights["u_total_inter"]

display(train_df_weights)

Посчитаем количество взаимодействий с определенным товаром (`item_count`)
на его основе расчитаем рейтинг товара (`item_rating`)


In [ ]:
# Все взаимодействия с каждым товаром
item_rating_df = (
    train_df_weights[["item_id", "ui_inter"]]
    .groupby("item_id")
    .sum()
    .rename(
        columns={
            "ui_inter": "item_count",
        }
    )
)

# Общий вес\рейтинг товара по всем пользователям
item_rating_df["item_rating"] = (
    item_rating_df["item_count"] / item_rating_df.shape[0]
)

# Отсортируем значения
item_rating_df = item_rating_df.reset_index().sort_values("item_rating", ascending=False)

item_rating_df

Сохраним таблицы в бинарник

In [ ]:
with open(data_path + "train_df_weights.dill", "wb") as f:
    dill.dump(train_df_weights, f)

with open(data_path + "item_rating.dill", "wb") as f:
    dill.dump(item_rating_df, f)

## 3. Мерджим таблицу с весами к основной

Теперь будем присоединять веса к общей таблице (где существуют данные о дате взаимодействия)

In [ ]:
with open(data_path + "train_df.dill", "rb") as f:
    train_df = dill.load(f)

with open(data_path + "train_df_weights.dill", "rb") as f:
    train_df_weights = dill.load(f)

In [ ]:
train_df = train_df.merge(train_df_weights, on=["user_id", "item_id"], how="left")

train_df

In [ ]:
with open(data_path + "train_df.dill", "wb") as f:
    dill.dump(train_df, f)

## 4. Добавляем новые фитчи к train_df

In [ ]:
# Загружаем таблицу
with open(data_path + "train_df.dill", "rb") as f:
    train_df = dill.load(f)

train_df

### Порядковых номера взаимодействия пользователя

In [ ]:
# порядковый номер взаимодействия пользователя
train_df["u_entry"] = train_df.groupby(["user_id"]).cumcount() + 1


# Подсчет порядковых номеров взаимодействия пользователя с 
# каждым конкретынм товаром
#
# Т.е. если взаимодействия с товарами идут в следующем порядке
# [1, 2, 1, 1, 2, 4]
#
# то результат будет следующим:
#
# cumcount([1, 2, 1, 1, 2, 4]) + 1 = [1, 1, 2, 3, 2, 1]
#
train_df["ui_entry"] = train_df.groupby(["user_id", "item_id"]).cumcount() + 1

train_df

### Кумулятивный и относительный веса

Добавим вес предмета в зависимости от номера вхождения этого предмета.
Условно, чем позже было взаимодействие с предметом, тем его вес больше.
Как это работает: допустим у нас есть расчитаем для $i$-го айтема в "корзине". 
Пусть длина корзины $l = 20$, предмет встречался n = 5 раз в этой корзине, тогда 
общий рейтинг этого предмета будет 
$$rating = \dfrac{n}{l} = \dfrac{5}{20} = 0.25.$$
Но также нам необходи кумулятивный вес, который рассчитывается для порядкового номера вхождения
айтема в корзину: т.е. для $j-го$ взаимодействия юзера с товаром кумулятивный вес будет следующий: 
$$cumWeight_j = \dfrac{j}{n} \cdot rating$$

Для рассматривоемого товара веса будут следующими: 
$$
\begin{gathered}
cumWeight_1 = \dfrac{1}{5} \cdot \dfrac{5}{20} = 0.05,\quad
cumWeight_2 = \dfrac{2}{5} \cdot \dfrac{5}{20} = 0.1,\quad
cumWeight_3 = \dfrac{3}{5} \cdot \dfrac{5}{20} = 0.15,\\
cumWeight_4 = \dfrac{4}{5} \cdot \dfrac{5}{20} = 0.2, \quad
cumWeight_5 = \dfrac{5}{5} \cdot \dfrac{5}{20} = 0.25.
\end{gathered}
$$


Так же добавим относительный вес товара (**rel_weight**), т.е. вес соответствующий товару 
при каждом новом взаимодействии пользователя с товаром

In [ ]:
# отношение порядкового номера взаимодействия пользователя 
# с конкретным товаром к общему числу взаимодействий пользователя
# с данным товаром
train_df["ui_entry_inter_ratio"] = train_df["ui_entry"] / train_df["ui_inter"]

# кумулятивный вес товара на момент просмотра
train_df["cum_weight"] = train_df["weight"] * train_df["ui_entry_inter_ratio"]


# вес (рейтинг) товара на момент просмотра
train_df["rel_weight"] = train_df["ui_entry"] / train_df["u_entry"]

train_df

In [ ]:
# Поменяем порядок следования столбцов
train_df = train_df[
    [
        "user_id",
        "item_id",
        "dt",
        "ui_inter",
        "u_total_inter",
        "ui_entry",
        "u_entry",
        "ui_entry_inter_ratio",
        "weight",
        "cum_weight",
        "rel_weight",
    ]
]

train_df

In [ ]:
with open(data_path + "train_df.dill", "wb") as f:
    dill.dump(train_df, f)

## 5. Разбиение train_df на base_model_data и ranker_data

In [ ]:
# Загружаем таблицу
with open(data_path + "train_df.dill", "rb") as f:
    train_df = dill.load(f)

train_df

> Под обучение модели второго уровня выделим 1/5 от данных идущих на train, т.е. ~8 часов.

In [ ]:
max_data = train_df["dt"].max()

# Дата разделяющая данные для трейна моделей
# первого и второго уровней
base_ranker_sep = max_data - pd.Timedelta(hours=8)

# Данные для обучения моделей первого уровня
# ranker_data = train_df[(train_df["dt"] >= base_ranker_sep)]
# Сразу сохраим в бинарник
with open(data_path + "ranker_data.dill", "wb") as f:
    dill.dump(train_df[(train_df["dt"] >= base_ranker_sep)], f)


# Данные для обучения модели второго уровня
# base_models_data = train_df[(train_df["dt"] < base_ranker_sep)]
# Сразу сохраим в бинарник
with open(data_path + "base_models_data.dill", "wb") as f:
    dill.dump(train_df[(train_df["dt"] < base_ranker_sep)], f)


## Выделим группы пользователей 

In [ ]:
# Загружаем таблицу
with open(data_path + "base_models_data.dill", "rb") as f:
    base_models_data = dill.load(f)

# Загружаем таблицу
with open(data_path + "ranker_data.dill", "rb") as f:
    ranker_data = dill.load(f)

# Загружаем таблицу
with open(data_path + "test_df.dill", "rb") as f:
    test_df = dill.load(f)


In [ ]:
# Уникальные айдишники пользователей в таблицах
base_users = base_models_data["user_id"].unique()
ranker_users = ranker_data["user_id"].unique()
test_users = test_df["user_id"].unique()

# Пользователи, которым надо выдавать пресказания для обучения ранкера,
# т.е. присутствуют и в base_models_data и в ranker_data (base to ranker users)
b2r_users = np.array(list((set(base_users) & set(ranker_users))))
display("b2r_users", b2r_users, b2r_users.shape)

# на оставшихся пользователях ранкер обучаться не будет
# на них просто не будет скоров
ranker_only_users = np.array(list(set(ranker_users) - set(base_users)))
display("ranker_only_users", ranker_only_users, ranker_only_users.shape)

# Пользователи из test_df, которым будут выданы
# таргетирвонные рекомондации
bNr2t_users = np.array(list((set(base_users) | set(ranker_users)) & set(test_users)))
display("bNr2t_users", bNr2t_users, bNr2t_users.shape)

# Пользователи, которые присутствуют только в test_df (cold_users)
test_only_users = np.array(list(set(test_users) - (set(base_users) | set(ranker_users))))
display("test_only_users", test_only_users, test_only_users.shape)